In [2]:
%cd ..

d:\Blockchain\data\orai\orchai_validator


In [3]:
from omegaconf import OmegaConf
from labeling.tools import psql_connect
import numpy as np

In [4]:
config = OmegaConf.load("config/etl.yaml")
cur = psql_connect(**config.dest).cursor()

## Validate

In [4]:
cur.execute(f"select distinct block_height from {config.dest.table} order by block_height;")
bh = cur.fetchall()
bh = np.array([b[0] for b in bh])

In [ ]:
bh.shape

In [ ]:
diff = np.diff(bh)
diff.sort()

from collections import Counter

c = Counter(diff)
c

## Block intervals

In [14]:
from labeling.upload import get_batch_intervals

vote_proposed_win_size = config.hp.etl.vote_proposed_win_size
combine_win_size = config.hp.etl.combine_win_size
label_win_size = config.hp.etl.label_win_size
batch_size = config.hp.upload.batch_size

for bs, be in get_batch_intervals(
    start_block=7063871,
    global_end_block=7303870,
    batch_size=batch_size,
    vote_proposed_win_size=vote_proposed_win_size,
    combine_win_size=combine_win_size,
    label_win_size=label_win_size,
    min_block=7049472
):
    rs = bs + vote_proposed_win_size - 1
    print(rs, be)

7063871 7199470
7098671 7234270
7133471 7269070
7168271 7303870


## Missing block:
```
7272731 7272972 | 241
7883969 7884210 | 241
8699979 8700001 | 22
9057005 9057016 | 11
9057065 9057076 | 11
```

In [ ]:
diff = np.diff(bh)
combine_win_size = config.hp.etl.combine_win_size

for i, d in enumerate(diff):
    if d != combine_win_size:
        print(bh[i], bh[i+1], "|", d)

## Total block

In [8]:
cur.execute(f"select min(block_height) from public.validator_block;")
print(cur.fetchall())

[(7049472,)]


In [9]:
config.hp.etl

{'accept_rate': 0.1, 'concentration_level': 0.9, 'vote_score': 2, 'propose_score': 6, 'A': 9, 'B': 4, 'C': 2, 'D': 4, 'vote_proposed_win_size': 14400, 'combine_win_size': 150, 'label_win_size': 100800}

In [10]:
print("total blocks:", ((9070703 - 100800 - 150) - (7049472 + 14400 - 1)) / 150 + 1)

total blocks: 12706.88
